In [6]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.getcwd()
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

%matplotlib inline 

/home/abhi/CS/utcs/fri/final-project


In [7]:
import elevator_panel

config = elevator_panel.Config
ELEVATOR_PANEL_DIR = os.path.join(ROOT_DIR, "elevator_panels")

In [8]:
dataset = elevator_panel.ElevatorPanelDataset()
dataset.load_elevator(ELEVATOR_PANEL_DIR, "train")

dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))


Image Count: 79
Class Count: 4
  0. BG                                                
  1. label                                             
  2. button                                            
  3. label_and_button                                  


In [10]:
# Load and display random samples
# image_ids = np.random.choice(dataset.image_ids, 4)
# for image_id in image_ids:
image_id = "IMG_1421"
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset.class_names)


TypeError: list indices must be integers or slices, not str